In [7]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('fake_reviews_dataset.csv')
df.head()

,category,rating,text,label
0,Home_and_Kitchen,5.0,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen,5.0,"love it, a great upgrade from the original. I...",1
2,Home_and_Kitchen,5.0,This pillow saved my back. I love the look and...,1
3,Home_and_Kitchen,1.0,"Missing information on how to use it, but it i...",1
4,Home_and_Kitchen,5.0,Very nice set. Good quality. We have had the s...,1


In [31]:
def preprocess_and_split_data(df):
    # Load stop words and initialize stemmer
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()

    # Preprocessing function: remove stop words and apply stemming
    def preprocess_text(text):
        words = text.split()
        filtered_words = [stemmer.stem(word) for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)

    # Split the data into training and testing sets (80% train, 20% test)
    X = df['text']  # Features (text data)
    y = df['label']  # Labels
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Apply preprocessing to the training and testing sets separately
    X_train = X_train.apply(preprocess_text)
    X_test = X_test.apply(preprocess_text)

    # Tokenize and pad sequences
    max_words = 10000  # Hardcoded maximum number of words
    max_len = 100      # Hardcoded maximum sequence length
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

    # Return the processed datasets and tokenizer
    return X_train_pad, X_test_pad, y_train, y_test, tokenizer

In [32]:
# Example usage for preprocessing
X_train, X_test, y_train, y_test, tokenizer = preprocess_and_split_data(df)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (32420, 100) (32420,)
Test shape: (8106, 100) (8106,)


### Base GRU Model

In [8]:
# Define GRU-based model
model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=100),  # Match tokenizer and padding
    GRU(32),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the GRU model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("GRU Test Accuracy:", accuracy)

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


811/811 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.5256 - loss: 0.6878 - val_accuracy: 0.7472 - val_loss: 0.5305
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.8233 - loss: 0.3987 - val_accuracy: 0.8823 - val_loss: 0.2776
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9203 - loss: 0.1972 - val_accuracy: 0.8894 - val_loss: 0.2682
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9457 - loss: 0.1361 - val_accuracy: 0.8917 - val_loss: 0.2865
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.9621 - loss: 0.1018 - val_accuracy: 0.8880 - val_loss: 0.3150
Epoch 6/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.9633 - loss: 0.0972 - val_accuracy: 0.8885 - val_loss: 0.3219
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8942 - loss: 0.2663
GRU Test Accuracy: 0.893905758857727


### GRU with Dropout Regularization 

In [10]:
# Define GRU-based model
model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=100),  # Match tokenizer and padding
    GRU(32, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the GRU model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("GRU Test Accuracy:", accuracy)

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


811/811 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.5165 - loss: 0.6917 - val_accuracy: 0.5242 - val_loss: 0.6829
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.5838 - loss: 0.6389 - val_accuracy: 0.8543 - val_loss: 0.3273
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.8849 - loss: 0.2854 - val_accuracy: 0.8826 - val_loss: 0.2749
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.9213 - loss: 0.2021 - val_accuracy: 0.8896 - val_loss: 0.2671
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.9393 - loss: 0.1622 - val_accuracy: 0.8886 - val_loss: 0.2828
Epoch 6/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.9510 - loss: 0.1341 - val_accuracy: 0.8883 - val_loss: 0.2995
Epoch 7/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9614 - loss: 0.1100 - val_accuracy: 0.8890 - val_loss: 0.3104
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8889 - loss: 0.2683
GRU Test Accuracy: 0.88921

### Increasing Embedding Dim and GRU from 32 to 64

In [14]:
# Define GRU-based model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),  # Match tokenizer and padding
    GRU(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the GRU model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 34s 41ms/step - accuracy: 0.5133 - loss: 0.6908 - val_accuracy: 0.7838 - val_loss: 0.4853
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 32s 40ms/step - accuracy: 0.8379 - loss: 0.3779 - val_accuracy: 0.8812 - val_loss: 0.2763
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 32s 39ms/step - accuracy: 0.9204 - loss: 0.2048 - val_accuracy: 0.8868 - val_loss: 0.2685
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 31s 38ms/step - accuracy: 0.9453 - loss: 0.1506 - val_accuracy: 0.8885 - val_loss: 0.2746
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 31s 38ms/step - accuracy: 0.9579 - loss: 0.1147 - val_accuracy: 0.8882 - val_loss: 0.3108
Epoch 6/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.9666 - loss: 0.0933 - val_accuracy: 0.8927 - val_loss: 0.3120
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8956 - loss: 0.2677
GRU Test Accuracy: 0.8953861594200134


### Bidirectional GRU

In [15]:
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64)),  # Wrap GRU in Bidirectional and increase units slightly
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 39s 47ms/step - accuracy: 0.7476 - loss: 0.4677 - val_accuracy: 0.8865 - val_loss: 0.2683
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9293 - loss: 0.1768 - val_accuracy: 0.9067 - val_loss: 0.2208
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - accuracy: 0.9575 - loss: 0.1157 - val_accuracy: 0.9047 - val_loss: 0.2372
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9710 - loss: 0.0803 - val_accuracy: 0.9024 - val_loss: 0.2579
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9795 - loss: 0.0603 - val_accuracy: 0.8988 - val_loss: 0.3199
254/254 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9002 - loss: 0.2397
Bidirectional GRU Test Accuracy: 0.9031581282615662


### Adding additional dense layer to Bidirectional GRU

In [16]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),  # New hidden layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 45ms/step - accuracy: 0.7386 - loss: 0.4637 - val_accuracy: 0.9013 - val_loss: 0.2403
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9352 - loss: 0.1628 - val_accuracy: 0.9126 - val_loss: 0.2113
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - accuracy: 0.9648 - loss: 0.0958 - val_accuracy: 0.9092 - val_loss: 0.2475
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - accuracy: 0.9764 - loss: 0.0638 - val_accuracy: 0.8998 - val_loss: 0.3154
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9821 - loss: 0.0498 - val_accuracy: 0.8994 - val_loss: 0.3291
254/254 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9100 - loss: 0.2309
Bidirectional GRU Test Accuracy: 0.9100666046142578


### Adding Dropout to the Above

In [20]:
from tensorflow.keras.layers import Dropout

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64, return_sequences=False)),
    Dropout(0.2),  # Regularizes GRU output
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 44ms/step - accuracy: 0.7273 - loss: 0.5022 - val_accuracy: 0.8927 - val_loss: 0.2509
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 40s 49ms/step - accuracy: 0.9263 - loss: 0.1800 - val_accuracy: 0.9028 - val_loss: 0.2334
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9553 - loss: 0.1165 - val_accuracy: 0.9041 - val_loss: 0.2388
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - accuracy: 0.9723 - loss: 0.0759 - val_accuracy: 0.9064 - val_loss: 0.2951
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9813 - loss: 0.0539 - val_accuracy: 0.8981 - val_loss: 0.3182
254/254 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8961 - loss: 0.2460
Bidirectional GRU Test Accuracy: 0.8994572162628174


#### Adding dropout seems to make it worse (I tried both 0.2 and 0.3)

In [22]:
# Extract the 'text' column from the DataFrame
texts = df['text']

# Initialize the tokenizer and fit it on the texts
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)

# Convert the texts to sequences of tokens
sequences = tokenizer.texts_to_sequences(texts)

# Calculate the lengths of each review in tokens
review_lengths = [len(seq) for seq in sequences]

# Calculate the average review length
average_length = sum(review_lengths) / len(review_lengths)
print("Average review length in tokens:", average_length)

Average review length in tokens: 32.007920840941615


### Reducing Learning Rate

In [23]:
from tensorflow.keras.layers import Dropout

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64, return_sequences=False)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 44ms/step - accuracy: 0.7012 - loss: 0.5093 - val_accuracy: 0.8846 - val_loss: 0.2628
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - accuracy: 0.9203 - loss: 0.1899 - val_accuracy: 0.9027 - val_loss: 0.2330
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9481 - loss: 0.1284 - val_accuracy: 0.9041 - val_loss: 0.2473
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - accuracy: 0.9683 - loss: 0.0880 - val_accuracy: 0.9053 - val_loss: 0.2796
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9733 - loss: 0.0710 - val_accuracy: 0.9021 - val_loss: 0.3343
254/254 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8986 - loss: 0.2417
Bidirectional GRU Test Accuracy: 0.8984702825546265


### Further Increasing Embedding Dim

In [26]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),  # New hidden layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 48s 56ms/step - accuracy: 0.7316 - loss: 0.4872 - val_accuracy: 0.8967 - val_loss: 0.2481
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.9323 - loss: 0.1734 - val_accuracy: 0.9098 - val_loss: 0.2182
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 45s 55ms/step - accuracy: 0.9631 - loss: 0.0953 - val_accuracy: 0.8988 - val_loss: 0.2650
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.9768 - loss: 0.0648 - val_accuracy: 0.9052 - val_loss: 0.2811
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 45s 56ms/step - accuracy: 0.9848 - loss: 0.0435 - val_accuracy: 0.9056 - val_loss: 0.3381
254/254 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9054 - loss: 0.2344
Bidirectional GRU Test Accuracy: 0.9031581282615662


### Adding Additonal GRU layer

In [27]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64, return_sequences=True)),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 80s 96ms/step - accuracy: 0.7558 - loss: 0.4478 - val_accuracy: 0.9052 - val_loss: 0.2379
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 83s 102ms/step - accuracy: 0.9349 - loss: 0.1661 - val_accuracy: 0.9098 - val_loss: 0.2248
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 84s 103ms/step - accuracy: 0.9621 - loss: 0.1038 - val_accuracy: 0.9007 - val_loss: 0.2701
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 81s 99ms/step - accuracy: 0.9687 - loss: 0.0860 - val_accuracy: 0.9056 - val_loss: 0.2888
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 80s 98ms/step - accuracy: 0.9806 - loss: 0.0576 - val_accuracy: 0.9005 - val_loss: 0.2995
254/254 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9017 - loss: 0.2378
Bidirectional GRU Test Accuracy: 0.9018011093139648


### Adding Barth Normalization

In [28]:
from keras.layers import BatchNormalization

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64)),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 36s 42ms/step - accuracy: 0.7558 - loss: 0.4565 - val_accuracy: 0.5193 - val_loss: 3.3787
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - accuracy: 0.9225 - loss: 0.1931 - val_accuracy: 0.7933 - val_loss: 0.5860
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 36s 44ms/step - accuracy: 0.9554 - loss: 0.1220 - val_accuracy: 0.8711 - val_loss: 0.3431
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 36s 44ms/step - accuracy: 0.9667 - loss: 0.0861 - val_accuracy: 0.8834 - val_loss: 0.3123
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 36s 44ms/step - accuracy: 0.9781 - loss: 0.0599 - val_accuracy: 0.8933 - val_loss: 0.3302
Epoch 6/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - accuracy: 0.9833 - loss: 0.0451 - val_accuracy: 0.8974 - val_loss: 0.3486
Epoch 7/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9866 - loss: 0.0383 - val_accuracy: 0.8837 - val_loss: 0.5110
254/254 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8826 - loss: 0.3192
Bidirectional G

### Testing with addiitonal GRU layer with driopout and also another dense layer

In [29]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping

# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)),  # Added GRU layer with dropout
    Bidirectional(GRU(64)),  # Original GRU layer
    Dense(64, activation='relu'),  # Added Dense layer
    Dropout(0.2),  # Dropout for regularization
    Dense(32, activation='relu'),  # Original Dense layer
    Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 103s 122ms/step - accuracy: 0.7382 - loss: 0.4794 - val_accuracy: 0.8940 - val_loss: 0.2447
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 118s 146ms/step - accuracy: 0.9247 - loss: 0.1831 - val_accuracy: 0.9064 - val_loss: 0.2346
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 166s 205ms/step - accuracy: 0.9546 - loss: 0.1150 - val_accuracy: 0.9011 - val_loss: 0.2585
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 170s 209ms/step - accuracy: 0.9693 - loss: 0.0802 - val_accuracy: 0.9105 - val_loss: 0.2518
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 167s 206ms/step - accuracy: 0.9783 - loss: 0.0603 - val_accuracy: 0.8976 - val_loss: 0.2745
254/254 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9044 - loss: 0.2498
Bidirectional GRU Test Accuracy: 0.9059955477714539


### Trying with a bigger val set

In [33]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),  # New hidden layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.35,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 51s 74ms/step - accuracy: 0.7135 - loss: 0.4913 - val_accuracy: 0.8949 - val_loss: 0.2484
Epoch 2/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 52s 80ms/step - accuracy: 0.9240 - loss: 0.1860 - val_accuracy: 0.9050 - val_loss: 0.2290
Epoch 3/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 53s 80ms/step - accuracy: 0.9583 - loss: 0.1061 - val_accuracy: 0.8999 - val_loss: 0.2612
Epoch 4/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 52s 79ms/step - accuracy: 0.9762 - loss: 0.0661 - val_accuracy: 0.9013 - val_loss: 0.2998
Epoch 5/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 53s 80ms/step - accuracy: 0.9812 - loss: 0.0521 - val_accuracy: 0.8940 - val_loss: 0.3205
254/254 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8999 - loss: 0.2445
Bidirectional GRU Test Accuracy: 0.900937557220459


### Trying diff batch sizes

In [35]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),  # New hidden layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 42s 95ms/step - accuracy: 0.6821 - loss: 0.5441 - val_accuracy: 0.8933 - val_loss: 0.2489
Epoch 2/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 44s 109ms/step - accuracy: 0.9256 - loss: 0.1831 - val_accuracy: 0.9084 - val_loss: 0.2235
Epoch 3/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 44s 108ms/step - accuracy: 0.9600 - loss: 0.1032 - val_accuracy: 0.9085 - val_loss: 0.2435
Epoch 4/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 45s 110ms/step - accuracy: 0.9759 - loss: 0.0683 - val_accuracy: 0.9045 - val_loss: 0.2981
Epoch 5/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 45s 110ms/step - accuracy: 0.9838 - loss: 0.0472 - val_accuracy: 0.8976 - val_loss: 0.3157
254/254 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9006 - loss: 0.2407
Bidirectional GRU Test Accuracy: 0.9029114246368408


In [38]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),  # New hidden layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.20,
    callbacks=[early_stopping]
)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print("Bidirectional GRU Test Accuracy:", accuracy)

Epoch 1/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 75s 45ms/step - accuracy: 0.6918 - loss: 0.5235 - val_accuracy: 0.8849 - val_loss: 0.2623
Epoch 2/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 73s 45ms/step - accuracy: 0.9061 - loss: 0.2280 - val_accuracy: 0.8987 - val_loss: 0.2467
Epoch 3/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 73s 45ms/step - accuracy: 0.9248 - loss: 0.1829 - val_accuracy: 0.9072 - val_loss: 0.2192
Epoch 4/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 73s 45ms/step - accuracy: 0.9405 - loss: 0.1491 - val_accuracy: 0.9139 - val_loss: 0.2108
Epoch 5/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 6559s 4s/step - accuracy: 0.9511 - loss: 0.1239 - val_accuracy: 0.9144 - val_loss: 0.2184
Epoch 6/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 11281s 7s/step - accuracy: 0.9601 - loss: 0.1058 - val_accuracy: 0.9155 - val_loss: 0.2364
Epoch 7/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 20772s 13s/step - accuracy: 0.9674 - loss: 0.0895 - val_accuracy: 0.9116 - val_loss: 0.2707
Epoch 8/20
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 9161s 6s/step - accuracy: 0.971